___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Авторские права принадлежат Pierian Data Inc.</em></center>
<center><em>Для дополнительной информации посетите наш сайт <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Регуляризация в SciKit-Learn

Ранее мы создали набор полиномиальных признаков и затем применили для них линейную регрессию. Но мы можем сделать модель ещё лучше, используя регуляризацию.

Регуляризация старается минимизировать метрику "сумма квадратов остатков" (RSS - residual sum of squares) *и* "функцию штрафа". Функция штрафа старается "наказывать" модели с очень большими коэффициентами. Некоторые методы регуляризации по сути приводят к тому, что некоторые неполезные признаки получают нулевые коэффициенты, то есть модель не будет использовать эти признаки.

Давайте рассмотрим два метода регуляризации - гребневая регрессия (или ридж-регрессия, или регрессия риджа, англ. ridge regression) и регрессия лассо. Мы будем использовать их в сочетании с полиномиальными признаками, поскольку регуляризация модели не так эффективна на исходном наборе признаков набора данных X.

## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Загрузка данных и подготовительные шаги

In [3]:
df = pd.read_csv('/home/rostya/projects/python/github/ai/Machine-Learning-Data-Science/UNZIP_ME_FOR_NOTEBOOKS_ML_RUS_V7/08-Linear-Regression-Models/Advertising.csv')
X = df.drop('sales',axis=1)
y = df['sales']

### Полиномиальная конвертация

In [4]:
from sklearn.preprocessing import PolynomialFeatures

In [5]:
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)

In [6]:
poly_features = polynomial_converter.fit_transform(X)

### Разбиение на обучающий и тестовый наборы данных -  Train | Test Split

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

----
----

## Масштабирование данных

Хотя в наших данных все признаки имеют один и тот же порядок величины (в $1000 долларов), это не всегда верно в других наборах данных. Поскольку математика регуляризации будет суммировать коэффициенты, очень важно стандартизировать признаки. Посмотрите теоретические видео-лекции для более подробного рассказа на эту тему, а также почему мы делаем **обучение (fit)** только на обучающих данных, и делаем **трансформацию** обоих наборов данных отдельно друг от друга.

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
# help(StandardScaler)

In [11]:
scaler = StandardScaler()

In [12]:
scaler.fit(X_train)

StandardScaler()

In [13]:
X_train = scaler.transform(X_train)

In [14]:
X_test = scaler.transform(X_test)

## Гребневая регрессия (Ridge Regression)

Посмотрите видео-лекции по этой теме - там мы рассказываем про гребневую регрессию и про выбор параметра alpha.

In [15]:
from sklearn.linear_model import Ridge

In [16]:
ridge_model = Ridge(alpha=10)

In [17]:
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [18]:
test_predictions = ridge_model.predict(X_test)

In [19]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [20]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [21]:
MAE

0.5774404204714163

In [22]:
RMSE

0.894638646131965

Какова точность на обучающей выборке? (Мы будем использовать это позже для сравнения)

In [23]:
# Производительность на обучающем наборе
train_predictions = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.5288348183025309

### Выбираем значение alpha с помощью кросс-валидации

Детали есть в видео-лекции.

In [24]:
from sklearn.linear_model import RidgeCV

In [25]:
# help(RidgeCV)

In [26]:
# Выбираем способ оценки модели: https://scikit-learn.org/stable/modules/model_evaluation.html
# Отрицательные значения RMSE. Все метрики следуют принципу "больше значит лучше".

# Смотрим все варианты: sklearn.metrics.SCORERS.keys()
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0),scoring='neg_mean_absolute_error')

In [27]:
# Чем больше значений alpha Вы укажете, тем дольше времени потребуется.
# К счастью наш набор данных достаточно маленький.
ridge_cv_model.fit(X_train,y_train)

RidgeCV(scoring='neg_mean_absolute_error')

In [28]:
ridge_cv_model.alpha_

0.1

In [29]:
test_predictions = ridge_cv_model.predict(X_test)

In [30]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [31]:
MAE

0.427377488434534

In [32]:
RMSE

0.6180719926938822

In [33]:
# Метрика MAE на обучающем наборе
train_predictions = ridge_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.30941321056539456

In [34]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])


-----

## Регрессия Лассо

In [35]:
from sklearn.linear_model import LassoCV

In [53]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html
lasso_cv_model = LassoCV(eps=0.001,n_alphas=100,cv=5, max_iter=1000000)

In [54]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, max_iter=1000000)

In [55]:
lasso_cv_model.alpha_

0.004943070909225831

In [56]:
test_predictions = lasso_cv_model.predict(X_test)

In [57]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [58]:
MAE

0.4335034618590074

In [59]:
RMSE

0.6063140748984036

In [60]:
# Метрика MAE на обучающем наборе
train_predictions = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.33070562256744884

In [61]:
lasso_cv_model.coef_

array([ 4.86023329,  0.12544598,  0.20746872, -4.99250395,  4.38026519,
       -0.22977201, -0.        ,  0.07267717, -0.        ,  1.77780246,
       -0.69614918, -0.        ,  0.12044132, -0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        ])

## Elastic Net

Elastic Net сочетает в себе ридж-регрессию и лассо-регрессию, чтобы взять лучшее из обеих регуляризаций!

In [45]:
from sklearn.linear_model import ElasticNetCV

In [47]:
elastic_cv_model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], eps=0.001, n_alphas=100, max_iter=1000000)

In [48]:
elastic_cv_model.fit(X_train, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000000)

In [51]:
elastic_cv_model.l1_ratio_  # Наилучшее значение

1.0

In [52]:
elastic_cv_model.alpha_

0.004943070909225831

In [62]:
test_predictions = elastic_cv_model.predict(X_test)

In [63]:
MAE = mean_absolute_error(y_test,test_predictions)

In [65]:
MAE

0.4335034618590074

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
